## Upload a paper

In [ ]:
import os
import sys
import json
import requests
from requests.auth import HTTPBasicAuth

# Optional (if reading from a .csv file)
import numpy as np

# Optional (if coordinates not in decimal degrees)
from astropy import units as u
from astropy.coordinates import SkyCoord

### 1. Let's define the specific API endpoint and load credentials
You should NOT save your passwords in Python documents. Instead it is safest to either set them as environment variables, for example, in an open shell terminal type: <code>export SLED_USERNAME=<your_SLED_username></code> and <code>export SLED_PASSWORD=<your_SLED_password></code>. You can also include these two commands in your shell profile file (e.g. ~/.bash_profile in Unix/Linux/Mac) or in an environment file.

In [ ]:
username, password = os.getenv('SLED_USERNAME'), os.getenv('SLED_PASSWORD')
if (not username) or (not password):
    print('You have not set your SLED username and password environment variables.')

### 2. Select lenses to include in the paper

All the lenses **should already exist in SLED**, if not, then follow the instructions on [how to inject lenses](https://sled.amnh.org/sled_howto/#how-to-inject-lenses). All lenses **must be public** as well. Matching with existing SLED lenses happens via their **RA and DEC** coordinates. Coordinates should be in decimal degrees and in the J2000 system. In this example we use dummy coordinates - one can read a list of RA and DEC from a .csv file, or a .json file with exported lenses from SLED. Finally, the [bibcode](https://en.wikipedia.org/wiki/Bibcode) of the paper must be known. SLED uses the bibcode to retrieve the paper and related information automatically from ADS.

In [ ]:
ras = [340.125800,172.964400]
decs = [3.358550,-12.532900]

## If reading from a .csv file use the following command
#ras,decs = np.genfromtxt('random_file.csv',dtype=None,skip_header=1,usecols=[3,4],delimiter=',',encoding=None,unpack=True)


## If the RA and DEC are not in decimal degrees then use the following loop
#for i in range(0,len(ras)):
#    c1 = SkyCoord(ras[i], unit=(u.hourangle, u.deg))
#    c2 = SkyCoord(decs[i], unit=(u.hourangle, u.deg))
#    ras[i] = c1
#    decs[i] = c2

## If reading from a .json file exported from SLED
#ras = []
#decs = []
#with open('lenses.json') as f:
#    lenses_tmp = json.load(f)
#for lens in lenses_tmp:
#    ras.append(float(lens["ra"]))
#    decs.append(float(lens["dec"]))

    
lenses = []

for i in range(0,len(ras)):
    lenses.append(
        {
            "ra": ras[i],
            "dec": decs[i],
            "discovery": True,        # This is to indicate a discovery paper
            "classification": False,  # This is to indicate a paper with a classification
            "model": False            # This is to indicate a paper with a model
        }
    )

### 3. Define the paper

We package the lenses and the bibcode of the paper into one object.

In [ ]:
paper = {
    'bibcode': 'some_bibcode',
    'lenses': lenses
}
print(paper)

### 4. Now we query the database

Use the requests package to send a POST request to the relevant API endpoint. The response is parsed as json and converted to a dictionary. There is no need to change the following lines.


In [ ]:
print("Uploading paper to the SLED server...")
r = requests.post("https://sled.amnh.org/api/upload-papers/",
                  data=json.dumps([paper]),
                  headers={'Content-Type':'application/json'},
                  auth=HTTPBasicAuth(username,password)
                  )
dbquery = json.loads(r.text)

### 5. Understanding the response

The response will either contain errors, e.g. if some of the lenses are not public, or a success message. If successful, the newly added paper will now appear in [SLED](https://sled.amnh.org/sled_papers/query) and in your [user profile](https://sled.amnh.org/users/profile#papers).

In [ ]:
print(dbquery)